In [9]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.express as px
from policyengine_core.charts import format_fig
import plotly.graph_objects as go


In [10]:
reform = Reform.from_dict({
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  }
}, country_id="us")

reform_taxable = Reform.from_dict({
  "gov.contrib.states.or.rebate.state_tax_exempt": {
    "2024-01-01.2100-12-31": True
  },
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  },
  "gov.contrib.ubi_center.basic_income.taxable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

In [11]:
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
reformed_taxable = Microsimulation(reform=reform_taxable)


In [12]:
def calculate_poverty_impact_by_age(baseline, reformed, year):
    state_codes = baseline.calc("state_code", map_to="person", period=year)
    age = baseline.calc("age", map_to="person", period=year)

    baseline_poverty = baseline.calc("in_poverty", map_to="person", period=year)
    reform_poverty = reformed.calc("in_poverty", map_to="person", period=year)

    results = {}
    age_groups = [
        (0, 18, "0-17"),
        (18, 65, "18-64"),
        (65, 200, "65+"),
        (0, 200, "Overall")
    ]

    for min_age, max_age, label in age_groups:
        if label == "Overall":
            mask = (state_codes == "OR")
        else:
            mask = (state_codes == "OR") & (age >= min_age) & (age < max_age)

        baseline_poverty_group = baseline_poverty[mask].mean()
        reform_poverty_group = reform_poverty[mask].mean()

        relative_poverty_reduction = (reform_poverty_group - baseline_poverty_group) / baseline_poverty_group
        results[label] = relative_poverty_reduction

    return results


In [13]:
# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame(columns=["year", "age_group", "relative_poverty_reduction", "relative_poverty_reduction_taxable"])


In [14]:
# Calculate for each year and add to the DataFrame
for year in range(2025, 2028):
    results_untaxed = calculate_poverty_impact_by_age(baseline, reformed, year)
    results_taxed = calculate_poverty_impact_by_age(baseline, reformed_taxable, year)

    for age_group in results_untaxed.keys():
        new_row = pd.DataFrame({
            "year": [year],
            "age_group": [age_group],
            "relative_poverty_reduction": [results_untaxed[age_group]],
            "relative_poverty_reduction_taxable": [results_taxed[age_group]]
        })
        all_results = pd.concat([all_results, new_row], ignore_index=True)


In [15]:
# Display the results
print(all_results)

    year age_group  relative_poverty_reduction  \
0   2025      0-17                   -0.392697   
1   2025     18-64                   -0.217906   
2   2025       65+                   -0.053603   
3   2025   Overall                   -0.222731   
4   2026      0-17                   -0.492473   
5   2026     18-64                   -0.267750   
6   2026       65+                   -0.090053   
7   2026   Overall                   -0.277636   
8   2027      0-17                   -0.482759   
9   2027     18-64                   -0.259736   
10  2027       65+                   -0.092882   
11  2027   Overall                   -0.271545   

    relative_poverty_reduction_taxable  
0                            -0.392697  
1                            -0.217906  
2                            -0.053603  
3                            -0.222731  
4                            -0.461682  
5                            -0.243466  
6                            -0.050389  
7                    

In [22]:
# Define colors for age groups
colors = {
    "0-17": "#003366",     # Dark blue
    "18-64": "#0066cc",    # Medium blue
    "65+": "#4d94ff",      # Light blue
    "Overall": "#99ccff",  # Very light blue
}

# Create the plot
fig = go.Figure()

# Store final y-values for label positioning
final_y_values = {}

# Initialize min and max y values
y_min, y_max = float('inf'), float('-inf')

# Add traces for each age group
for age_group in colors.keys():
    data = all_results[all_results['age_group'] == age_group]
    
    # Taxed scenario
    taxed_trace = go.Scatter(
        x=data['year'], 
        y=data['relative_poverty_reduction_taxable'],
        mode='lines',
        name=f'{age_group} (Taxed)',
        line=dict(color=colors[age_group], width=2),
        showlegend=False
    )
    fig.add_trace(taxed_trace)
    final_y_values[f"{age_group} (Taxed)"] = taxed_trace.y[-1]
    
    # Update y_min and y_max
    y_min = min(y_min, min(taxed_trace.y))
    y_max = max(y_max, max(taxed_trace.y))
    
    # Untaxed scenario
    untaxed_trace = go.Scatter(
        x=data['year'], 
        y=data['relative_poverty_reduction'],
        mode='lines',
        name=f'{age_group} (Untaxed)',
        line=dict(color=colors[age_group], width=2, dash='dash'),
        showlegend=False
    )
    fig.add_trace(untaxed_trace)
    final_y_values[f"{age_group} (Untaxed)"] = untaxed_trace.y[-1]
    
    # Update y_min and y_max
    y_min = min(y_min, min(untaxed_trace.y))
    y_max = max(y_max, max(untaxed_trace.y))

# Add some padding to the y-axis range
y_range = y_max - y_min
y_min -= 0.1 * y_range
y_max += 0.1 * y_range

# Function to adjust label positions
def adjust_label_positions(positions, min_gap=0.02):
    sorted_items = sorted(positions.items(), key=lambda x: x[1])
    adjusted = {}
    for i, (key, value) in enumerate(sorted_items):
        if i == 0:
            adjusted[key] = value
        else:
            prev_key = sorted_items[i-1][0]
            if value - adjusted[prev_key] < min_gap:
                adjusted[key] = adjusted[prev_key] + min_gap
            else:
                adjusted[key] = value
    return adjusted

# Adjust label positions
adjusted_positions = adjust_label_positions(final_y_values)

# Update layout
fig.update_layout(
    title='Oregon Rebate Impact on Poverty by Age Group and Federal Taxation Over Time',
    xaxis_title='Year',
    yaxis_title='Poverty Reduction (%)',
    height=650,
    width=750,
    margin=dict(r=80, l=50, b=70, t=100),  # Slightly reduced right margin
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.02
    )
)

# Update x-axis
fig.update_xaxes(
    tickvals=[2025, 2026, 2027],
    ticktext=["2025", "2026", "2027"],
    range=[2024.8, 2027.15]  # Slightly reduced x-axis range
)

# Update y-axis
fig.update_yaxes(tickformat='.0%', range=[-0.5, 0])

# Update hover template
fig.update_traces(
    hovertemplate='Year: %{x}<br>Age Group: %{data.name}<br>Poverty Reduction: %{y:.2%}<extra></extra>'
)

# Add labels closer to the lines with adjusted positions
for label, y_pos in adjusted_positions.items():
    fig.add_annotation(
        x=2027.05,  # Moved closer to the end of the lines
        y=y_pos,
        text=label,
        showarrow=False,
        xanchor="left",
        yanchor="middle",
        font=dict(size=8, color=colors[label.split()[0]])
    )

# Apply the format_fig function
fig = format_fig(fig)

fig.show()